In [50]:
import sympy as sp
from moment import *
from latex_helper import *

# Running Example (example 1)

In [22]:
import time
import sympy as sp

###############################################################################
# Running Example
###############################################################################

def example_running():
    # SDE in 2D:
    #   dX_t = -X_t dt + dW_t^(1)
    #   dY_t = (-2 Y_t + X_t + X_t^2) dt + X_t dW_t^(2)
    #
    # State variables: x1 = X, x2 = Y
    x1, x2 = sp.symbols('x1 x2')
    vars_ = (x1, x2)

    b_vec = [
        -x1,
        -sp.Integer(2)*x2 + x1 + x1**2
    ]

    sigma_mat = [
        [sp.Integer(1), sp.Integer(0)],
        [sp.Integer(0), x1]
    ]

    # Target moment: m_{0,2}(t) = E[Y_t^2] = E[x2^2]
    alpha = (0, 2)

    # ---- Timing: closure (Algorithm 1) ----
    t0 = time.perf_counter()
    S, A_mat, c_vec = moment_closure_algorithm(b_vec, sigma_mat, alpha, vars_)
    t1 = time.perf_counter()
    closure_time = t1 - t0

    print("\n=== running example ===")
    print(f"Size of S: |S| = {len(S)}")
    print(f"Time for obtaining closure S: {closure_time:.6f} seconds")

    print("Index set S (multi-indices and corresponding monomials):")
    for s in pretty_print_S(S, vars_):
        print("  ", s)

    # print("\nA matrix (size {}x{}):".format(*A_mat.shape))
    # sp.pprint(A_mat)
    # print("\nc vector:")
    # sp.pprint(c_vec)

    t = sp.symbols('t', real=True)

    # Initial condition (X_0, Y_0) = (0, 0) ⇒ all moments in S are 0 at t=0
    m0_vec = sp.zeros(len(S), 1)

    # ---- Timing: solving ODE system ----
    t2 = time.perf_counter()
    m_t = solve_moment_system(A_mat, c_vec, m0_vec, t)
    t3 = time.perf_counter()
    ode_time = t3 - t2

    print(f"\nTime for solving ODE system: {ode_time:.6f} seconds")

    # m_{0,2}(t) is the first component of m(t) because we started S with alpha=(0,2)
    m_0_2_t = sp.simplify(m_t[0, 0])

    print("\nSolution m_{0,2}(t) = E[Y_t^2]:")
    sp.pprint(m_0_2_t)

    # -------- LaTeX output --------
    print("\nLaTeX: index set S")
    print(latex_index_set(S))

    print("\nLaTeX: ODE system")
    print(latex_moment_ode_system(S, A_mat, c_vec))

    print("\nLaTeX: solution for all moments in S")
    print(latex_moment_solutions(S, m_t))

    print("\nLaTeX: single moment m_{0,2}(t)")
    print("$" + latex_single_moment(S, m_t, alpha) + "$")


example_running()



=== running example ===
Size of S: |S| = 8
Time for obtaining closure S: 0.005530 seconds
Index set S (multi-indices and corresponding monomials):
   (0, 2) -> x2**2
   (2, 1) -> x1**2*x2
   (2, 0) -> x1**2
   (1, 1) -> x1*x2
   (4, 0) -> x1**4
   (3, 0) -> x1**3
   (0, 1) -> x2
   (1, 0) -> x1

Time for solving ODE system: 0.129062 seconds

Solution m_{0,2}(t) = E[Y_t^2]:
⎛    2                        2⋅t      4⋅t       t    ⎞  -4⋅t
⎝18⋅t  + 24⋅t - 3⋅(2⋅t + 11)⋅ℯ    + 8⋅ℯ    + 16⋅ℯ  + 9⎠⋅ℯ    
─────────────────────────────────────────────────────────────
                             24                              

LaTeX: index set S
S = \{m_{0,2}, m_{2,1}, m_{2,0}, m_{1,1}, m_{4,0}, m_{3,0}, m_{0,1}, m_{1,0}\}

LaTeX: ODE system
\begin{aligned}
\dot{m_{0,2}}(t) = - 4 m_{0,2}(t) + 2 m_{2,1}(t) + m_{2,0}(t) + 2 m_{1,1}(t) \\
\dot{m_{2,1}}(t) = - 4 m_{2,1}(t) + m_{4,0}(t) + m_{3,0}(t) + m_{0,1}(t) \\
\dot{m_{2,0}}(t) = 1 - 2 m_{2,0}(t) \\
\dot{m_{1,1}}(t) = m_{2,0}(t) - 3 m_{1,1}(t) +

In [33]:
import time
import sympy as sp

###############################################################################
# ou-env Example
###############################################################################

def example_OUenv():
    # SDE in 2D:
    #   dX_t = -X_t dt + dW_t^(1)
    #   dY_t = (-2 Y_t + X_t + X_t^2) dt + X_t dW_t^(2)
    #
    # State variables: x1 = X, x2 = Y
    x1, x2 = sp.symbols('x1 x2')
    vars_ = (x1, x2)

    b_vec = [
        -x1,
        -sp.Integer(2)*x2 + x1 + x1**2
    ]

    sigma_mat = [
        [sp.Integer(1), 0],
        [0, x1]
    ]

    # target multi-index
    alpha = (0, 5)

    # ---- Timing: closure (Algorithm 1) ----
    t0 = time.perf_counter()
    S, A_mat, c_vec = moment_closure_algorithm(b_vec, sigma_mat, alpha, vars_)
    t1 = time.perf_counter()
    closure_time = t1 - t0

    print("\n=== OU-env example ===")
    print(f"Size of S: |S| = {len(S)}")
    print(f"Time for obtaining closure S: {closure_time:.6f} seconds")

    print("Index set S (multi-indices and corresponding monomials):")
    for s in pretty_print_S(S, vars_):
        print("  ", s)

    # print("\nA matrix (size {}x{}):".format(*A_mat.shape))
    # sp.pprint(A_mat)
    # print("\nc vector:")
    # sp.pprint(c_vec)

    t = sp.symbols('t', real=True)

    # Initial condition (X_0, Y_0) = (0, 0) ⇒ all moments in S are 0 at t=0
    m0_vec = sp.zeros(len(S), 1)

    # ---- Timing: solving ODE system ----
    t2 = time.perf_counter()
    m_t = solve_moment_system(A_mat, c_vec, m0_vec, t)
    t3 = time.perf_counter()
    ode_time = t3 - t2

    print(f"\nTime for solving ODE system: {ode_time:.6f} seconds")

    # target moment
    idx = S.index(alpha)
    m_alpha = sp.simplify(m_t[idx, 0])

    print("\nSolution m_{alpha}(t) = E[X^alpha]:")
    sp.pprint(m_alpha)

    # -------- LaTeX output --------
    print("\nLaTeX: index set S")
    print(latex_index_set(S))

    # print("\nLaTeX: ODE system for the moments")
    # print(latex_moment_ode_system(S, A_mat, c_vec))

    # print("\nLaTeX: solution for all moments in S")
    # print(latex_moment_solutions(S, m_t))

    print("\nLaTeX: E[X^alpha]")
    print("$" + latex_single_moment(S, m_t, alpha) + "$")

example_OUenv()


=== OU-env example ===
Size of S: |S| = 35
Time for obtaining closure S: 0.042617 seconds
Index set S (multi-indices and corresponding monomials):
   (0, 5) -> x2**5
   (2, 4) -> x1**2*x2**4
   (2, 3) -> x1**2*x2**3
   (1, 4) -> x1*x2**4
   (4, 3) -> x1**4*x2**3
   (4, 2) -> x1**4*x2**2
   (3, 3) -> x1**3*x2**3
   (0, 4) -> x2**4
   (4, 1) -> x1**4*x2
   (3, 2) -> x1**3*x2**2
   (0, 3) -> x2**3
   (6, 2) -> x1**6*x2**2
   (6, 1) -> x1**6*x2
   (5, 2) -> x1**5*x2**2
   (6, 0) -> x1**6
   (5, 1) -> x1**5*x2
   (2, 2) -> x1**2*x2**2
   (1, 3) -> x1*x2**3
   (5, 0) -> x1**5
   (2, 1) -> x1**2*x2
   (1, 2) -> x1*x2**2
   (8, 1) -> x1**8*x2
   (8, 0) -> x1**8
   (7, 1) -> x1**7*x2
   (7, 0) -> x1**7
   (4, 0) -> x1**4
   (3, 1) -> x1**3*x2
   (0, 2) -> x2**2
   (3, 0) -> x1**3
   (0, 1) -> x2
   (10, 0) -> x1**10
   (9, 0) -> x1**9
   (2, 0) -> x1**2
   (1, 1) -> x1*x2
   (1, 0) -> x1

Time for solving ODE system: 2.979980 seconds

Solution m_{alpha}(t) = E[X^alpha]:
⎛              5       

# Case study 1: Consensus System

In [45]:
import time
import sympy as sp

def example_concensus_system():
    # 2D SDE:
    #   dX1_t = ( - 2 X1_t + X2_t) dt +  X1_t dW_t^(1)
    #   dX2_t = ( X1_t - 2 X2_t) dt +  X2_t dW_t^(2)
    #
    # State variables
    x1, x2 = sp.symbols('x1 x2')
    vars_ = (x1, x2)

    # Drift b(x)
    b_vec = [
        - sp.Integer(2)*x1 + x2,
         x1 - sp.Integer(2)*x2
    ]

    # Diffusion sigma(x): diagonal, independent Brownian motions
    sigma_mat = [
        [ x1, sp.Integer(0)],
        [sp.Integer(0), x2]
    ]

    # Target monomial
    alpha = (1, 1)

    # ---- Timing: closure (Algorithm 1) ----
    t0 = time.perf_counter()
    S, A_mat, c_vec = moment_closure_algorithm(b_vec, sigma_mat, alpha, vars_)
    t1 = time.perf_counter()
    closure_time = t1 - t0

    print("\n=== concensus system example ===")
    print(f"Size of S: |S| = {len(S)}")
    print(f"Time for obtaining closure S: {closure_time:.6f} seconds")

    print("Index set S (multi-indices and corresponding monomials):")
    for s in pretty_print_S(S, vars_):
        print("  ", s)

    print("\nA matrix (size {}x{}):".format(*A_mat.shape))
    sp.pprint(A_mat)
    print("\nc vector:")
    sp.pprint(c_vec)

    t = sp.symbols('t', real=True)

    # # Initial state X_0 = (0,0)  ⇒ all moments in S start from 0
    # m0_vec = sp.zeros(len(S), 1)

    # Initial state X_0 = (1,0)
    x10 = sp.Integer(1)
    x20 = sp.Integer(0)
    m0_list = []
    for beta in S:
        # m_beta(0) = 1^{beta1} * 0^{beta2}
        val = (x10 ** beta[0]) * (x20 ** beta[1])
        m0_list.append(val)
    m0_vec = sp.Matrix(m0_list)

    # ---- Timing: solving ODE system ----
    t2 = time.perf_counter()
    m_t = solve_moment_system(A_mat, c_vec, m0_vec, t)
    t3 = time.perf_counter()
    ode_time = t3 - t2

    print(f"\nTime for solving ODE system: {ode_time:.6f} seconds")

    # m_{0,2}(t) corresponds to monomial x2^2
    idx = S.index(alpha)
    m_0_2_t = sp.simplify(m_t[idx, 0])

    print("\nSolution m_alpha:")
    sp.pprint(m_0_2_t)

    # -------- LaTeX output --------
    print("\nLaTeX: index set S")
    print(latex_index_set(S))

    print("\nLaTeX: ODE system for the moments")
    print(latex_moment_ode_system(S, A_mat, c_vec))

    print("\nLaTeX: solution for all moments in S")
    print(latex_moment_solutions(S, m_t))

    print("\nLaTeX: E[X_alpha]")
    print("$" + latex_single_moment(S, m_t, alpha) + "$")


example_concensus_system()



=== concensus system example ===
Size of S: |S| = 3
Time for obtaining closure S: 0.007752 seconds
Index set S (multi-indices and corresponding monomials):
   (1, 1) -> x1*x2
   (2, 0) -> x1**2
   (0, 2) -> x2**2

A matrix (size 3x3):
⎡-4  1   1 ⎤
⎢          ⎥
⎢2   -3  0 ⎥
⎢          ⎥
⎣2   0   -3⎦

c vector:
⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

Time for solving ODE system: 0.232147 seconds

Solution m_alpha:
                  -t⋅(√17 + 7) 
                  ─────────────
    ⎛ √17⋅t    ⎞        2      
√17⋅⎝ℯ      - 1⎠⋅ℯ             
───────────────────────────────
              17               

LaTeX: index set S
S = \{m_{1,1}, m_{2,0}, m_{0,2}\}

LaTeX: ODE system for the moments
\begin{aligned}
\dot{m_{1,1}}(t) = - 4 m_{1,1}(t) + m_{2,0}(t) + m_{0,2}(t) \\
\dot{m_{2,0}}(t) = 2 m_{1,1}(t) - 3 m_{2,0}(t) \\
\dot{m_{0,2}}(t) = 2 m_{1,1}(t) - 3 m_{0,2}(t)
\end{aligned}

LaTeX: solution for all moments in S
\begin{aligned}
m_{1,1}(t) = \frac{\sqrt{17} \left(e^{\sqrt{17} t} - 1\right) e^{- \frac{t \l

# Vehicle platoon

In [46]:
import time
import sympy as sp

def example_vehicle_platoon_simple():
    # State variables ordered as (p1, v1, p2, v2)
    p1, v1, p2, v2 = sp.symbols('p1 v1 p2 v2')
    vars_ = (p1, v1, p2, v2)

    # Parameters (instantiated as in the text)
    d_des = 0

    # Spacing error s = (p1 - p2)
    s = p1 - p2 

    # Drift b(x):
    #   dp1 = v1 dt
    #   dv1 = (-a1 v1 + u1) dt + sigma1 dW^1, with a1=1, u1=1, sigma1=1
    #   dp2 = v2 dt
    #   dv2 = (-a2 v2 + (v1 - 1)^2) dt + sigma2 dW^2, a2=1, k=1, sigma2=1
    b_vec = [
        v1,                           # dp1
        -v1 + 1,                      # dv1
        v2,                           # dp2
        -v2 + sp.Rational(1, 2) + (v1 - 1)**2 ,     # dv2
        # -v2 + s + (v1 - 1)**2  
    ]

    # Diffusion matrix sigma(x) (4×2):
    # noise only in v1 (W^1) and v2 (W^2), both with coefficient 1
    sigma_mat = [
        [0, 0],   # dp1
        [0, 0],   # dv1
        [0, 0],   # dp2
        [0, sp.Rational(1, 10)],   # dv2
    ]

    # Initial state (p1, v1, p2, v2) = (1, 0, 0, 0)
    p1_0, v1_0, p2_0, v2_0 = 1, 0, 0, 0

    # We want moments of p1 and p2:
    targets = [
        # ((1, 0, 0, 0), "p_1"),  # monomial p1
        # ((0, 0, 1, 0), "p_2"),  # monomial p2
        # ((2, 0, 0, 0), "p_1^2"),  # monomial p1^2
        ((0, 0, 2, 0), "p_2^2"),  # monomial p2^2
        # ((1, 0, 1, 0), "p_1p_2"),  # monomial p1p2
    ]

    t = sp.symbols('t', real=True)

    for alpha, label in targets:
        # --------- closure S (Algorithm 1) ----------
        t0 = time.perf_counter()
        S, A_mat, c_vec = moment_closure_algorithm(b_vec, sigma_mat, alpha, vars_)
        t1 = time.perf_counter()

        print(f"\n=== Nonlinear vehicle platoon ({label}) ===")
        print(f"Target monomial: {label}")
        print(f"Size of S: |S| = {len(S)}")
        print(f"Time for obtaining closure S: {t1 - t0:.6f} seconds")

        print("Index set S (multi-indices and corresponding monomials):")
        for s_str in pretty_print_S(S, vars_):
            print("  ", s_str)

        print("\nA matrix (size {}x{}):".format(*A_mat.shape))
        sp.pprint(A_mat)
        print("\nc vector:")
        sp.pprint(c_vec)

        # Initial moment vector from deterministic initial state
        m0_list = []
        for beta in S:
            val = (p1_0 ** beta[0]) * (v1_0 ** beta[1]) \
                  * (p2_0 ** beta[2]) * (v2_0 ** beta[3])
            m0_list.append(val)
        m0_vec = sp.Matrix(m0_list)

        # --------- solve ODE system ----------
        t2 = time.perf_counter()
        m_t = solve_moment_system(A_mat, c_vec, m0_vec, t)
        t3 = time.perf_counter()
        print(f"\nTime for solving ODE system: {t3 - t2:.6f} seconds")

        # Extract desired moment E[p1(t)] or E[p2(t)]
        idx = S.index(alpha)
        moment_expr = sp.simplify(m_t[idx, 0])

        print(f"\nSolution for E[{label}(t)]:")
        sp.pprint(moment_expr)

        # --------- optional LaTeX output ----------
        print("\nLaTeX: index set S")
        print(latex_index_set(S))

        print("\nLaTeX: ODE system")
        print(latex_moment_ode_system(S, A_mat, c_vec))

        print("\nLaTeX: solution for all moments in S")
        print(latex_moment_solutions(S, m_t))

        print(f"\nLaTeX: E[{label}(t)]")
        print("$" + latex_single_moment(S, m_t, alpha) + "$")

# To run:
example_vehicle_platoon_simple()



=== Nonlinear vehicle platoon (p_2^2) ===
Target monomial: p_2^2
Size of S: |S| = 13
Time for obtaining closure S: 0.017685 seconds
Index set S (multi-indices and corresponding monomials):
   (0, 0, 2, 0) -> p2**2
   (0, 0, 1, 1) -> p2*v2
   (0, 2, 1, 0) -> p2*v1**2
   (0, 1, 1, 0) -> p2*v1
   (0, 0, 1, 0) -> p2
   (0, 0, 0, 2) -> v2**2
   (0, 2, 0, 1) -> v1**2*v2
   (0, 1, 0, 1) -> v1*v2
   (0, 0, 0, 1) -> v2
   (0, 4, 0, 0) -> v1**4
   (0, 3, 0, 0) -> v1**3
   (0, 2, 0, 0) -> v1**2
   (0, 1, 0, 0) -> v1

A matrix (size 13x13):
⎡0  2   0   0    0   0   0   0   0   0   0    0    0 ⎤
⎢                                                    ⎥
⎢0  -1  1   -2  3/2  1   0   0   0   0   0    0    0 ⎥
⎢                                                    ⎥
⎢0  0   -2  2    0   0   1   0   0   0   0    0    0 ⎥
⎢                                                    ⎥
⎢0  0   0   -1   1   0   0   1   0   0   0    0    0 ⎥
⎢                                                    ⎥
⎢0  0   0   0    0   0  

# Gene example

In [36]:
import time
import sympy as sp

def example_gene_expression_network():
    # 5D gene regulatory network SDE (from the figure in the paper)
    x1, x2, x3, x4, x5 = sp.symbols('x1 x2 x3 x4 x5')
    vars_ = (x1, x2, x3, x4, x5)

    # Drift b(x)
    b_vec = [
        -x1 + 1,
        sp.Rational(12, 10)* x1 - sp.Rational(8, 10) * x2,
        x2 - sp.Rational(7, 10) * x3 + sp.Rational(2, 10) * x1**2,
        sp.Rational(9, 10) * x3 - sp.Rational(6, 10) * x4 + sp.Rational(1, 10) * x1 * x2,
        sp.Rational(8, 10)* x4 - sp.Rational(5, 10) * x5 + sp.Rational(15, 100) * x3**2 + sp.Rational(5, 100) * x1**3,
    ]

    # Diffusion sigma(x): diagonal, one independent Brownian motion per component
    sigma_mat = [
        [sp.Rational(1, 2), sp.Integer(0), sp.Integer(0), sp.Integer(0), sp.Integer(0)],
        [sp.Integer(0), sp.Rational(3, 10) * x1 +sp.Rational(2, 5), 0, 0, 0],
        [sp.Integer(0),sp.Integer(0), sp.Rational(1, 2) * x2 + sp.Rational(1, 10) * x1**2, sp.Integer(0), sp.Integer(0)],
        [sp.Integer(0), sp.Integer(0), sp.Integer(0), sp.Rational(2, 5) * x3 +sp.Rational(1, 5)* x2**2, sp.Integer(0)],
        [sp.Integer(0), sp.Integer(0),sp.Integer(0), sp.Integer(0), sp.Rational(3, 10) * x4 + sp.Rational(1, 10) * x3**2 + sp.Rational(5, 100) * x1**3],
    ]

    # Target moment
    alpha = (1, 0, 0, 0, 2)

    # ---- Timing: closure (Algorithm 1) ----
    t0 = time.perf_counter()
    S, A_mat, c_vec = moment_closure_algorithm(b_vec, sigma_mat, alpha, vars_)
    t1 = time.perf_counter()
    closure_time = t1 - t0

    print("\n=== Gene expression network example ===")
    print(f"Size of S: |S| = {len(S)}")
    print(f"Time for obtaining closure S: {closure_time:.6f} seconds")

    print("Index set S (multi-indices and corresponding monomials):")
    for s in pretty_print_S(S, vars_):
        print("  ", s)

    # print("\nA matrix (size {}x{}):".format(*A_mat.shape))
    # sp.pprint(A_mat)
    # print("\nc vector:")
    # sp.pprint(c_vec)

    t = sp.symbols('t', real=True)

    # Initial state X_0 = 0 => all moments in S start from 0
    m0_vec = sp.zeros(len(S), 1)

    # ---- Timing: solving ODE system ----
    t2 = time.perf_counter()
    m_t = solve_moment_system(A_mat, c_vec, m0_vec, t)
    t3 = time.perf_counter()
    ode_time = t3 - t2

    print(f"\nTime for solving ODE system: {ode_time:.6f} seconds")

    # target moment
    idx = S.index(alpha)
    m_alpha = sp.simplify(m_t[idx, 0])

    print("\nSolution m_{alpha}(t) = E[X^alpha]:")
    sp.pprint(m_alpha)

    # -------- LaTeX output --------
    print("\nLaTeX: index set S")
    print(latex_index_set(S))

    # print("\nLaTeX: ODE system for the moments")
    # print(latex_moment_ode_system(S, A_mat, c_vec))

    # print("\nLaTeX: solution for all moments in S")
    # print(latex_moment_solutions(S, m_t))

    print("\nLaTeX: E[X^alpha]")
    print("$" + latex_single_moment(S, m_t, alpha) + "$")

example_gene_expression_network()


=== Gene expression network example ===
Size of S: |S| = 115
Time for obtaining closure S: 0.178775 seconds
Index set S (multi-indices and corresponding monomials):
   (1, 0, 0, 0, 2) -> x1*x5**2
   (7, 0, 0, 0, 0) -> x1**7
   (4, 0, 2, 0, 0) -> x1**4*x3**2
   (4, 0, 0, 1, 0) -> x1**4*x4
   (4, 0, 0, 0, 1) -> x1**4*x5
   (1, 0, 4, 0, 0) -> x1*x3**4
   (1, 0, 2, 1, 0) -> x1*x3**2*x4
   (1, 0, 2, 0, 1) -> x1*x3**2*x5
   (1, 0, 0, 2, 0) -> x1*x4**2
   (1, 0, 0, 1, 1) -> x1*x4*x5
   (0, 0, 0, 0, 2) -> x5**2
   (6, 0, 0, 0, 0) -> x1**6
   (5, 0, 0, 0, 0) -> x1**5
   (8, 0, 0, 0, 0) -> x1**8
   (6, 1, 0, 0, 0) -> x1**6*x2
   (6, 0, 1, 0, 0) -> x1**6*x3
   (4, 2, 0, 0, 0) -> x1**4*x2**2
   (4, 1, 1, 0, 0) -> x1**4*x2*x3
   (3, 0, 2, 0, 0) -> x1**3*x3**2
   (2, 0, 2, 0, 0) -> x1**2*x3**2
   (5, 1, 0, 0, 0) -> x1**5*x2
   (4, 0, 1, 0, 0) -> x1**4*x3
   (3, 0, 0, 1, 0) -> x1**3*x4
   (2, 0, 0, 1, 0) -> x1**2*x4
   (3, 0, 0, 0, 1) -> x1**3*x5
   (2, 0, 0, 0, 1) -> x1**2*x5
   (5, 0, 2, 0, 0) -> 

# Non Pro-solvable System

In [39]:
import time
import sympy as sp

def example_2d_non_prosolvable_x1_2_x2_2():
    # 2D SDE:
    #   dX1_t = -X1_t^3 dt + sqrt(2) * X1_t^2 dW_t^(1)
    #   dX2_t = -X2_t   dt + 1 * dW_t^(2)
    #
    # Initial condition: (X1_0, X2_0) = (1, 1)

    x1, x2 = sp.symbols('x1 x2')
    vars_ = (x1, x2)

    # Parameters λ = 1, σ = 1; use SymPy sqrt for sqrt(2)
    lam = sp.Integer(1)
    sig = sp.Integer(1)

    # Drift
    b_vec = [
        -x1**3,          # b1(x) = -x1^3
        -lam * x2        # b2(x) = -λ x2, λ=1
    ]

    # Diffusion matrix (2×2) with independent Brownian motions
    sigma_mat = [
        [sp.sqrt(2) * x1**2, 0],   # σ1(x) = sqrt(2) x1^2
        [0, sig]                   # σ2(x) = σ, σ=1
    ]

    # Target monomial x1^2 x2^2  ->  m_{2,2}(t)
    alpha = (2, 2)

    # ---------- closure S (Algorithm 1) ----------
    t0 = time.perf_counter()
    S, A_mat, c_vec = moment_closure_algorithm(b_vec, sigma_mat, alpha, vars_)
    t1 = time.perf_counter()

    print("\n=== 2D non–pro-solvable SDE example (moment x1^2 x2^2) ===")
    print(f"Size of S: |S| = {len(S)}")
    print(f"Time for obtaining closure S: {t1 - t0:.6f} seconds")

    print("Index set S (multi-indices and corresponding monomials):")
    for s_str in pretty_print_S(S, vars_):
        print("  ", s_str)

    print("\nA matrix (size {}x{}):".format(*A_mat.shape))
    sp.pprint(A_mat)
    print("\nc vector:")
    sp.pprint(c_vec)

    t = sp.symbols('t', real=True)

    # Initial condition (X1_0, X2_0) = (1, 1)
    x1_0 = sp.Integer(1)
    x2_0 = sp.Integer(1)

    # Build initial moment vector m(0)
    m0_list = []
    for beta in S:
        val = (x1_0 ** beta[0]) * (x2_0 ** beta[1])
        m0_list.append(val)
    m0_vec = sp.Matrix(m0_list)

    # ---------- solve ODE system ----------
    t2 = time.perf_counter()
    m_t = solve_moment_system(A_mat, c_vec, m0_vec, t)
    t3 = time.perf_counter()
    print(f"\nTime for solving ODE system: {t3 - t2:.6f} seconds")

    # Extract m_{2,2}(t) = E[X1_t^2 X2_t^2]
    idx = S.index(alpha)
    m_2_2_t = sp.simplify(m_t[idx, 0])

    print("\nSolution m_{2,2}(t) = E[X_{1,t}^2 X_{2,t}^2]:")
    sp.pprint(m_2_2_t)

    # ---------- LaTeX outputs (optional) ----------
    print("\nLaTeX: index set S")
    print(latex_index_set(S))

    print("\nLaTeX: ODE system")
    print(latex_moment_ode_system(S, A_mat, c_vec))

    print("\nLaTeX: solution for all moments in S")
    print(latex_moment_solutions(S, m_t))

    print("\nLaTeX: E[X_{1,t}^2 X_{2,t}^2]")
    print("$" + latex_single_moment(S, m_t, alpha) + "$")

# To run:
example_2d_non_prosolvable_x1_2_x2_2()



=== 2D non–pro-solvable SDE example (moment x1^2 x2^2) ===
Size of S: |S| = 2
Time for obtaining closure S: 0.008516 seconds
Index set S (multi-indices and corresponding monomials):
   (2, 2) -> x1**2*x2**2
   (2, 0) -> x1**2

A matrix (size 2x2):
⎡-2  1⎤
⎢     ⎥
⎣0   0⎦

c vector:
⎡0⎤
⎢ ⎥
⎣0⎦

Time for solving ODE system: 0.026816 seconds

Solution m_{2,2}(t) = E[X_{1,t}^2 X_{2,t}^2]:
     -2⋅t
1   ℯ    
─ + ─────
2     2  

LaTeX: index set S
S = \{m_{2,2}, m_{2,0}\}

LaTeX: ODE system
\begin{aligned}
\dot{m_{2,2}}(t) = - 2 m_{2,2}(t) + m_{2,0}(t) \\
\dot{m_{2,0}}(t) = 0
\end{aligned}

LaTeX: solution for all moments in S
\begin{aligned}
m_{2,2}(t) = \frac{1}{2} + \frac{e^{- 2 t}}{2} \\
m_{2,0}(t) = 1
\end{aligned}

LaTeX: E[X_{1,t}^2 X_{2,t}^2]
$m_{2,2}(t) = \frac{1}{2} + \frac{e^{- 2 t}}{2}$


# Oscillator

In [ ]:
import time
import sympy as sp

def example_oscillator():
    # State variables X_t = (x1, x2, x3)
    x1, x2, x3 = sp.symbols('x1 x2 x3')
    vars_ = (x1, x2, x3)

    # Use exact rationals instead of floats:
    # 0.3 = 3/10, 0.8 = 4/5, 0.2 = 1/5, 0.5 = 1/2.
    b_vec = [
        x2,  # dX1_t = X2_t dt
        -sp.Rational(3, 10)*x2 - x1 + sp.Rational(4, 5)*x3**2,  # dX2 drift
        -x3  # dX3 drift
    ]

    # Diffusion matrix sigma(x) (3×2), W^(1) and W^(2) independent
    sigma_mat = [
        [0, 0],                                  # dX1 has no noise
        [sp.Rational(1, 5)*x2, 0],               # 0.2 X2 dW^(1)
        [0, sp.Rational(1, 2)]                   # 0.5 dW^(2)
    ]

    # Target monomial x1^2 x2^3  ->  m_{2,3,0}(t)
    alpha = (0, 1, 2)

    # ---------- closure S (Algorithm 1) ----------
    t0 = time.perf_counter()
    S, A_mat, c_vec = moment_closure_algorithm(b_vec, sigma_mat, alpha, vars_)
    t1 = time.perf_counter()

    print("\n=== osclliator SDE example  ===")
    print(f"Size of S: |S| = {len(S)}")
    print(f"Time for obtaining closure S: {t1 - t0:.6f} seconds")

    print("Index set S (multi-indices and corresponding monomials):")
    for s_str in pretty_print_S(S, vars_):
        print("  ", s_str)

    print("\nA matrix (size {}x{}):".format(*A_mat.shape))
    sp.pprint(A_mat)
    print("\nc vector:")
    sp.pprint(c_vec)

    t = sp.symbols('t', real=True)

    # Initial condition (X1_0, X2_0, X3_0) = (0, 0, 0)
    x1_0 = sp.Integer(0)
    x2_0 = sp.Integer(0)
    x3_0 = sp.Integer(0)

    # Build initial moment vector m(0)
    m0_list = []
    for beta in S:
        val = (x1_0 ** beta[0]) * (x2_0 ** beta[1]) * (x3_0 ** beta[2])
        m0_list.append(val)
    m0_vec = sp.Matrix(m0_list)

    # ---------- solve ODE system ----------
    t2 = time.perf_counter()
    m_t = solve_moment_system(A_mat, c_vec, m0_vec, t)
    t3 = time.perf_counter()
    print(f"\nTime for solving ODE system: {t3 - t2:.6f} seconds")

    # Extract m_{2,3,0}(t) = E[X1_t^2 X2_t^3]
    idx = S.index(alpha)
    m_2_3_0_t = sp.simplify(m_t[idx, 0])

    print("\nSolution m_{2,3,0}(t) = E[X_{1,t}^2 X_{2,t}^3]:")
    sp.pprint(m_2_3_0_t)

    # ---------- LaTeX outputs (optional) ----------
    print("\nLaTeX: index set S")
    print(latex_index_set(S))

    print("\nLaTeX: ODE system")
    print(latex_moment_ode_system(S, A_mat, c_vec))

    print("\nLaTeX: solution for all moments in S")
    print(latex_moment_solutions(S, m_t))

    print("\nLaTeX: E[X_{1,t}^2 X_{2,t}^3]")
    print("$" + latex_single_moment(S, m_t, alpha) + "$")

# To run:
example_oscillator()



=== 3D SDE example (moment x1^2 x2^3) ===
Size of S: |S| = 6
Time for obtaining closure S: 0.009566 seconds
Index set S (multi-indices and corresponding monomials):
   (0, 1, 2) -> x2*x3**2
   (1, 0, 2) -> x1*x3**2
   (0, 1, 0) -> x2
   (0, 0, 4) -> x3**4
   (1, 0, 0) -> x1
   (0, 0, 2) -> x3**2

A matrix (size 6x6):
⎡-23                           ⎤
⎢────  -1   1/4   4/5   0    0 ⎥
⎢ 10                           ⎥
⎢                              ⎥
⎢ 1    -2    0     0   1/4   0 ⎥
⎢                              ⎥
⎢ 0    0   -3/10   0   -1   4/5⎥
⎢                              ⎥
⎢ 0    0     0    -4    0   3/2⎥
⎢                              ⎥
⎢ 0    0     1     0    0    0 ⎥
⎢                              ⎥
⎣ 0    0     0     0    0   -2 ⎦

c vector:
⎡ 0 ⎤
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎣1/4⎦

Time for solving ODE system: 2.367202 seconds

Solution m_{2,3,0}(t) = E[X_{1,t}^2 X_{2,t}^3]:
⎛                                                53⋅t                         